## **PROJECT PYTHON :** Whith 66% accuracy


**Import Libraries :**

  * Keras → Permet la construction de réseaux de neurones et l'entrainement de
modèle sur des données
  * Scikit-learn → Permet l'utilisation d'outils pour la classification, la régression, le clustering, etc
  * Matplotlib → Permet de tracer et visualiser les données sous forme de graphiques
  

In [ ]:
from keras.datasets import cifar100
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Flatten, Dense, BatchNormalization, Dropout
from keras.optimizers import RMSprop
from sklearn.model_selection import train_test_split
import keras
import matplotlib.pyplot as plt

**Load and Preprocess the Data**


* **x_train** ← données des images utilisées pour l'entraînement
* **y_train** ← classes correspondant aux images d'entraînement


* **x_val** ← données des images utilisées pour la validation
* **y_val** ← classes correspondant aux images de validation

**Normalisation**
<br>*Permet de faciliter l'apprentissage, réduire les pertes et améliorer les performances du modèle.*
* **astype('float32')** → convertie les données des images *(pixels)* en valeurs flottente *(float32)*
* **/255** → ramène les valeurs RGB des images dans l'interval [0,1] *(pour une meilleure efficacité du model)*

**Transformation**
<br>*Permet de faciliter la classification et l'utilisation dans un modèle d'apprentissage automatique.*
* **to_categorical()** → transforme les classes sous forme de vecteur binaire (0 ou 1)

In [ ]:
# Load the CIFAR-100 dataset
(x_train, y_train), (x_test, y_test) = cifar100.load_data()

# Preprocess the data
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
num_classes = 100
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

**Data augmentation**

*Permet de générer de nouvelles données en appliquant des transformations aléatoires aux images provenant du dataset.*
* rotation_range → rotations aléatoires jusqu'à 20 degrés
* horizontal_flip → retournement horizontal aléatoire des images


In [ ]:
# Data augmentation
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

**Split the Training data**

In [ ]:
# Split the training data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

**CNN architecture**

In [ ]:
# Create a Sequential model
model = Sequential()

# Block 1
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(32, 32, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))

# Block 2
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Block 3
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

# Flatten and fully connected layers
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(100, activation='softmax'))

**Compile the model**

In [ ]:

# Compile the model with a learning rate schedule
initial_learning_rate = 0.001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=10000, decay_rate=0.9, staircase=True
)
opt = RMSprop(learning_rate=lr_schedule)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

# Implement Early Stopping
early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


**Train the model**




In [ ]:

# Train the model
epochs = 100
batch_size = 64  # Define your batch size
steps_per_epoch = len(x_train) // batch_size

class SaveBestAccuracyModel(Callback):
    def __init__(self, filepath, monitor='val_accuracy', mode='max', verbose=1):
        super(SaveBestAccuracyModel, self).__init__()
        self.filepath = filepath
        self.monitor = monitor
        self.mode = mode
        self.verbose = verbose
        self.best_accuracy = -1

    def on_epoch_end(self, epoch, logs=None):
        current_accuracy = logs.get(self.monitor)

        if current_accuracy is None:
            warnings.warn("Accuracy metric not found, skipping saving model.")
            return

        if (self.mode == 'max' and current_accuracy > self.best_accuracy) or \
           (self.mode == 'min' and current_accuracy < self.best_accuracy):
            if self.verbose > 0:
                print(f"\nEpoch {epoch + 1}: {self.monitor} improved from {self.best_accuracy} to {current_accuracy}. Saving model.")
            self.best_accuracy = current_accuracy
            self.model.save(self.filepath, overwrite=True)
        else:
            if self.verbose > 0:
                print(f"\nEpoch {epoch + 1}: {self.monitor} did not improve from {self.best_accuracy}. Model not saved.")

# Specify the path where you want to save the best model
best_model_path = 'best_model.h5'

# Create an instance of the custom callback
save_best_accuracy_model = SaveBestAccuracyModel(filepath=best_model_path)

# Include the callback in the list of callbacks during model training
callbacks_list = [save_best_accuracy_model]
# Train the model with callbacks
history = model.fit(
    datagen.flow(x_train, y_train, batch_size=batch_size),
    steps_per_epoch=steps_per_epoch,
    epochs=epochs,
    validation_data=(x_val, y_val),
    callbacks=callbacks_list
)

# Load the best model for evaluation
best_model = keras.models.load_model(best_model_path)


**Evaluate the model**

On évalue les performances du modèle sur l'ensemble de données de validation (images et classes)

In [ ]:
# Evaluate the best model on the test data
test_loss, test_accuracy = best_model.evaluate(x_test, y_test, batch_size=batch_size)
print('Best Model - Test loss:', test_loss)
print('Best Model - Test accuracy:', test_accuracy * 100)

**Display the result**

On affiche sous forme de 2 graphiques les résultats (pertes et précision en fonction du nombre d'épochs) obtenu pendant l'entrainement et la validation du modèle

In [ ]:
# Plot the training history
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], linestyle='dashed', label='val')
plt.title('Loss vs. Epochs')
plt.ylabel('Loss')
plt.xlabel('Epochs')
plt.legend(loc='center right')

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], color='green', label='train')
plt.plot(history.history['val_accuracy'], linestyle='dashed', color='red', label='val')
plt.title('Accuracy vs. Epochs')
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(loc='center right')

plt.tight_layout()
plt.show()
